In [1]:
import torch
import torch.nn as nn

class Custom3DCNN(nn.Module):
    def __init__(self, num_classes):
        super(Custom3DCNN, self).__init__()
        self.conv1 = nn.Conv3d(in_channels=3, out_channels=32, kernel_size=(3, 3, 3), stride=1, padding=1)
        self.bn1 = nn.BatchNorm3d(32)
        self.relu = nn.ReLU()
        self.pool1 = nn.MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2))

        self.conv2 = nn.Conv3d(32, 64, kernel_size=(3, 3, 3), stride=1, padding=1)
        self.bn2 = nn.BatchNorm3d(64)
        self.pool2 = nn.MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2))

        self.conv3 = nn.Conv3d(64, 128, kernel_size=(3, 3, 3), stride=1, padding=1)
        self.bn3 = nn.BatchNorm3d(128)
        self.pool3 = nn.MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2))

        self.fc1 = nn.Linear(128 * 3 * 14 * 14, 512)  # Adjust based on input size
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.pool1(self.relu(self.bn1(self.conv1(x))))
        x = self.pool2(self.relu(self.bn2(self.conv2(x))))
        x = self.pool3(self.relu(self.bn3(self.conv3(x))))
        x = x.view(x.size(0), -1)  # Flatten
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [2]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

class VideoDataset(Dataset):
    def __init__(self, data_path, split, transform=None):
        self.data_path = data_path
        self.split = split
        self.samples = []
        self.transform = transform

        # Load the mapping from class names to indices
        classes = [cls for cls in os.listdir(os.path.join(data_path, split)) if os.path.isdir(os.path.join(data_path, split, cls))]
        self.class_to_idx = {cls: idx for idx, cls in enumerate(sorted(classes))}

        # Collect file paths and labels
        for class_name in classes:
            class_dir = os.path.join(data_path, split, class_name)
            for file in os.listdir(class_dir):
                if file.endswith('.npy'):
                    # Use the class-to-index mapping
                    label = self.class_to_idx[class_name]
                    self.samples.append((os.path.join(class_dir, file), label))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        video_path, label = self.samples[idx]
        frames = np.load(video_path).astype(np.float32) / 255.0  # Normalize
        frames = torch.tensor(frames).permute(3, 0, 1, 2)  # Convert to (C, T, H, W)
        return frames, label



In [3]:
train_dataset = VideoDataset(data_path=r"C:\ProjectX\processed_data", split='train')
val_dataset = VideoDataset(data_path=r"C:\ProjectX\processed_data", split='val')

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False)




In [4]:
def train(model, loader, optimizer, criterion, device):
    model.train()
    total_loss, correct = 0, 0
    for inputs, labels in loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        correct += (outputs.argmax(1) == labels).sum().item()

    accuracy = 100 * correct / len(loader.dataset)
    print(f"Train Loss: {total_loss:.4f}, Accuracy: {accuracy:.2f}%")
    return total_loss, accuracy


def evaluate(model, loader, criterion, device):
    model.eval()
    total_loss, correct = 0, 0
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            total_loss += loss.item()
            correct += (outputs.argmax(1) == labels).sum().item()

    accuracy = 100 * correct / len(loader.dataset)
    print(f"Val Loss: {total_loss:.4f}, Accuracy: {accuracy:.2f}%")
    return total_loss, accuracy


In [17]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Custom3DCNN(num_classes=101).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10
for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    train(model, train_loader, optimizer, criterion, device)
    evaluate(model, val_loader, criterion, device)


Epoch 1/10
Train Loss: 12156.6787, Accuracy: 1.12%
Val Loss: 1230.7017, Accuracy: 1.22%
Epoch 2/10
Train Loss: 11058.2063, Accuracy: 1.04%
Val Loss: 1229.9757, Accuracy: 1.22%
Epoch 3/10
Train Loss: 11051.7417, Accuracy: 1.14%
Val Loss: 1229.7660, Accuracy: 1.22%
Epoch 4/10
Train Loss: 11050.5126, Accuracy: 1.11%
Val Loss: 1229.6743, Accuracy: 1.22%
Epoch 5/10
Train Loss: 11068.0549, Accuracy: 1.13%
Val Loss: 1229.7237, Accuracy: 1.22%
Epoch 6/10
Train Loss: 11049.8458, Accuracy: 1.09%
Val Loss: 1229.7040, Accuracy: 1.22%
Epoch 7/10
Train Loss: 11049.8844, Accuracy: 1.19%
Val Loss: 1229.7080, Accuracy: 1.22%
Epoch 8/10
Train Loss: 11049.7605, Accuracy: 1.23%
Val Loss: 1229.7134, Accuracy: 1.22%
Epoch 9/10
Train Loss: 11049.8052, Accuracy: 1.05%
Val Loss: 1229.7051, Accuracy: 1.22%
Epoch 10/10
Train Loss: 11049.6733, Accuracy: 1.15%
Val Loss: 1229.7000, Accuracy: 1.22%


In [18]:
test_dataset = VideoDataset(data_path=r"C:\ProjectX\processed_data", split='test')
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

evaluate(model, test_loader, criterion, device)


Val Loss: 1533.6598, Accuracy: 1.24%


(1533.659755706787, 1.2387387387387387)

In [19]:
torch.save(model.state_dict(), r"C:\Users\acer\custom_3dcnn.pth")


In [20]:
import torch
import os

# Define the directory where the model will be saved
save_dir = r"C:\Users\acer\OneDrive\Documents\Dev Work\AI\ProjectX\ProjectX\models"
os.makedirs(save_dir, exist_ok=True)

# Full path for the model
save_path = os.path.join(save_dir, "custom_3dcnn_full.pth")

# Save the full model
try:
    torch.save(model, save_path)
    print(f"Model saved successfully at: {save_path}")
except Exception as e:
    print(f"Failed to save the model: {e}")


Model saved successfully at: C:\Users\acer\OneDrive\Documents\Dev Work\AI\ProjectX\ProjectX\models\custom_3dcnn_full.pth


In [22]:
import torch
import os

# Save to a simpler path
save_dir = r"C:\temp\models"
os.makedirs(save_dir, exist_ok=True)

save_path = os.path.join(save_dir, "custom_3dcnn_full.pth")

# Save the full model
try:
    torch.save(model, save_path)
    print(f"Model saved successfully at: {save_path}")
except Exception as e:
    print(f"Failed to save the model: {e}")


Model saved successfully at: C:\temp\models\custom_3dcnn_full.pth
